### GOAL

Build one tiny backend resource using correct thinking order.

In [1]:
"""
    1. Resources -> User
"""

'\n    1. Resources -> User\n'

In [2]:
"""
    2. Database Model
"""

import uuid
from sqlmodel import Field, SQLModel

class User(SQLModel, table=True):
    __tablename__="users"
    uid: uuid.UUID | None = Field(default=uuid.uuid4, primary_key=True)
    email : str = Field(..., unique=True, nullable=False, index=True)
    full_name: str = Field(...)
    hash_psswd: str = Field(..., nullable=False)

In [3]:
"""
    3. Schema
"""
import uuid
from pydantic import BaseModel

class UserCreate(BaseModel):
    email: str
    full_name: str
    hash_psswd: str
    
class UserRead(BaseModel):
    uid: uuid.UUID
    email: str
    full_name: str

In [4]:
"""
    4. Service
"""

from fastapi import HTTPException, status
from sqlmodel import select
from sqlalchemy.ext.asyncio import AsyncSession


class UserService:
    async def create(self, payload: UserCreate, session: AsyncSession):
        statement = select(User).where(User.email == payload.email)
        result = await session.execute(statement)
        user = result.scalar_one_or_none()
        
        if user:
            return {"error":"User already exists"}
        
        new_user = User(email = payload.email,
                        full_name = payload.full_name,
                        password = payload.hash_psswd)
        
        
        session.add(new_user)
        await session.commit()
        await session.refresh(new_user)
        
        return new_user
    
    
    async def get_user_by_id(self, user_id: uuid.UUID, session: AsyncSession):
        statement = select(User).where(User.uid == user_id)
        result = await session.execute(statement)
        user = result.scalar_one_or_none()
        
        if not user:
            raise ValueError("User not found")
        
        return user
    
    
    async def get_all_users(self, session: AsyncSession):
        statement = select(User)
        resutl = await session.execute(statement)
        users = resutl.scalars().all()
        
        if not users:
            raise ValueError("Users not found")
        
        return users

In [5]:
def get_session(ab):
    pass

In [6]:
"""
    4. Routes (REST Endpoints)
"""


from fastapi import FastAPI, status, Depends, HTTPException
from sqlalchemy.ext.asyncio import AsyncSession

app = FastAPI()
user_service = UserService()

'''Creating User'''
@app.post("/user/create", response_model=UserRead, status_code=status.HTTP_201_CREATED)
async def create_user(payload: UserCreate, session: AsyncSession = Depends(get_session)):
    try:
        user = await user_service.create(payload, session)
        return user
    
    except ValueError as e:
        raise HTTPException(status_code=status.HTTP_400_BAD_REQUEST, detail=str(e))



'''Fetch all users'''
app.get("/users/", response_model=list[UserRead], status_code=status.HTTP_200_OK)
async def get_all_user(session: AsyncSession = Depends(get_session)):
    try:
        all_user = await user_service.get_all_users(session)
        return all_user
    
    except Exception as e:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Users not found")



'''Fetching the user'''    
@app.get("/user/{user_id}", response_model=UserRead, status_code=status.HTTP_200_OK)
async def get_user(user_id: uuid.UUID, session: AsyncSession = Depends(get_session)):
    user = await user_service.get_user_by_id(user_id, session)
    
    if user is None:
        return []
    
    return user